# 🎨 MyVLM-Art 概念嵌入训练

在 Google Colab 上训练概念嵌入模型。

⚠️ **要求**: GPU 运行时 (T4 或更高)

## 1. 检查 GPU

In [ ]:
import torch
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"显存: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# 检查 CUDA 版本
!nvcc --version

## 2. 安装依赖 (修复 bitsandbytes)

In [ ]:
# 卸载旧版本 bitsandbytes
!pip uninstall -y bitsandbytes

# 安装兼容 Colab CUDA 12.x 的 bitsandbytes
!pip install bitsandbytes>=0.41.0 --no-cache-dir

# 验证 bitsandbytes 安装
!python -m bitsandbytes

In [ ]:
%%capture
# 安装其他依赖
!pip install accelerate>=0.28.0
!pip install transformers>=4.37.2
!pip install peft>=0.2.0
!pip install open-clip-torch
!pip install timm>=0.9.16
!pip install sentence-transformers>=2.3.1
!pip install omegaconf>=2.3.0
!pip install pyrallis>=0.3.1
!pip install loguru>=0.7.0
!pip install wandb>=0.16.0  # wandb for training logging
print("✅ 依赖安装完成")

In [ ]:
# 重启 runtime 以应用 bitsandbytes (必须!)
import os
os.kill(os.getpid(), 9)

⚠️ **重启后从这里继续** ⬇️

In [ ]:
# 验证 bitsandbytes 正常工作
import bitsandbytes as bnb
print(f"✅ bitsandbytes 版本: {bnb.__version__}")

## 3. Clone 代码仓库

In [ ]:
import os

# TODO: 替换为你的 GitHub 仓库地址
REPO_URL = "https://github.com/YOUR_USERNAME/MyVLM_art.git"

# 如果是私有仓库，使用 token
# REPO_URL = "https://<TOKEN>@github.com/YOUR_USERNAME/MyVLM_art.git"

if not os.path.exists("/content/MyVLM_art"):
    !git clone {REPO_URL}
    
os.chdir("/content/MyVLM_art")
print(f"📁 当前目录: {os.getcwd()}")

### (备选) 从 Google Drive 复制

In [ ]:
# 如果代码在 Google Drive 上，运行这个 cell
# from google.colab import drive
# drive.mount('/content/drive')
# !cp -r "/content/drive/MyDrive/MyVLM_art" /content/
# import os
# os.chdir("/content/MyVLM_art")

## 4. 验证文件结构

In [ ]:
from pathlib import Path

# 检查必要文件
required_files = [
    "artifacts/prototypes_artist_trained.pt",
    "artifacts/prototypes_style_trained.pt",
    "artifacts/prototypes_genre_trained.pt",
    "artifacts/concept_signals_artist.json",
    "artifacts/concept_signals_style.json",
    "artifacts/concept_signals_genre.json",
    "artifacts/synth_targets.csv",
    "data/dataset/wikiart_5artists_dataset.json",
]

print("📁 检查必要文件:")
all_exist = True
for f in required_files:
    exists = Path(f).exists()
    status = "✅" if exists else "❌"
    print(f"  {status} {f}")
    if not exists:
        all_exist = False

if all_exist:
    print("\n✅ 所有文件就绪!")
else:
    print("\n❌ 缺少文件，请检查仓库")

## 4.5 配置 Wandb (可选)

设置 wandb API key 以监控训练 loss。可以通过以下任一方式设置：
1. **环境变量** `WANDB_API_KEY` (推荐)
2. 在 `~/.wandb_api_key` 文件中写入 key
3. 直接在 config 中传入 `wandb_api_key`

获取 API key: https://wandb.ai/authorize

In [ ]:
import os

# 方法 1: 从环境变量设置 (推荐在 Colab Secrets 中设置)
# 在 Colab 左侧点击 🔑 图标，添加 Secret: WANDB_API_KEY
from google.colab import userdata
try:
    os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY')
    print("✅ Wandb API key 已从 Colab Secrets 读取")
except:
    print("⚠️ 未在 Colab Secrets 中找到 WANDB_API_KEY")

# 方法 2: 手动设置 (不推荐，不要提交到 Git)
# os.environ["WANDB_API_KEY"] = "your-api-key-here"

# 方法 3: 写入文件 (在本地运行时可用)
# from pathlib import Path
# (Path.home() / ".wandb_api_key").write_text("your-api-key-here")

# 验证 wandb 安装
try:
    import wandb
    print(f"✅ wandb 版本: {wandb.__version__}")
except ImportError:
    print("❌ wandb 未安装，运行: pip install wandb")

## 5. 运行训练

In [ ]:
# 直接运行训练脚本
!python concept_graph/concept_embeddings/train_concept_embedding.py

### (可选) 自定义参数运行

In [ ]:
# 如果需要调整参数，可以直接在这里运行
import sys
sys.path.insert(0, "/content/MyVLM_art")

from pathlib import Path
import torch
from myvlm.common import VLMType, PersonalizationTask, VLM_TO_LAYER
from configs.myvlm_art_config import MyVLMArtConfig
from vlms.llava_wrapper import LLaVAWrapper
from myvlm.myllava import MyLLaVA
from concept_graph.concept_embeddings.trainer import MultiTokenEmbeddingTrainer

# 可以调整的参数
cfg = MyVLMArtConfig(
    concept_name="dataset",
    concept_identifier="painting",
    vlm_type=VLMType.LLAVA,
    personalization_task=PersonalizationTask.CAPTIONING,
    output_root=Path("./outputs"),
    data_root=Path("./data"),
    optimization_steps=100,       # 训练步数
    learning_rate=1.0,
    batch_size=4,                 # 根据 GPU 显存调整 (T4 建议 2-4)
    reg_lambda=0.0075,
    device='cuda',
    torch_dtype=torch.float16,
    threshold=0.75,
    max_tokens_per_concept=6,
    max_concepts_per_sample=3,
    backoff_delta=0.05,
    val_subset_n=3,
    max_reason_tokens=64,
    grad_accum_steps=4,
    attn_reg_interval=4,
    # Wandb 配置 (从环境变量读取 key，无需在此设置)
    use_wandb=True,                          # 启用 wandb 日志
    wandb_project="myvlm-art-training",      # wandb 项目名
    wandb_run_name=None,                     # 留空则自动生成
)

print("🔄 加载 LLaVA 模型...")
vlm = LLaVAWrapper(device=cfg.device, torch_dtype=cfg.torch_dtype)

print("🔧 初始化 MyLLaVA...")
myvlm = MyLLaVA(vlm, layer=VLM_TO_LAYER[cfg.vlm_type], concept_name=cfg.concept_name, cfg=cfg)

print("🚀 开始训练...")
trainer = MultiTokenEmbeddingTrainer(cfg=cfg, myvlm=myvlm, dataset_builder=None)
checkpoints = trainer.train()

# 保存
from datetime import datetime
from zoneinfo import ZoneInfo
out_dir = cfg.output_root / cfg.concept_name / f"seed_{cfg.seed}"
out_dir.mkdir(parents=True, exist_ok=True)
ts = datetime.now(ZoneInfo('America/New_York')).strftime("%Y%m%d_%H%M%S")
save_path = out_dir / f"checkpoints_{cfg.concept_name}_seed_{cfg.seed}_{ts}.pt"
torch.save(checkpoints, save_path)
print(f"\n💾 检查点已保存: {save_path}")

## 6. 查看训练结果

In [ ]:
from pathlib import Path
import torch

output_dir = Path("outputs/dataset/seed_42")
ckpts = sorted(output_dir.glob("checkpoints_*.pt"))

if ckpts:
    latest = ckpts[-1]
    print(f"📦 最新检查点: {latest}")
    
    data = torch.load(latest, map_location="cpu")
    print(f"📊 包含 {len(data)} 个迭代的检查点")
    
    for k, v in data.items():
        if isinstance(v, dict):
            keys_shape = v.get('keys', torch.tensor([])).shape
            values_shape = v.get('values', torch.tensor([])).shape
            print(f"  迭代 {k}: keys={keys_shape}, values={values_shape}")
else:
    print("❌ 未找到检查点文件")

## 7. 保存到 Google Drive

In [ ]:
from google.colab import drive
import shutil
from pathlib import Path

drive.mount('/content/drive')

# 复制输出文件
save_dir = Path("/content/drive/MyDrive/MyVLM_art_outputs")
save_dir.mkdir(parents=True, exist_ok=True)

shutil.copytree("outputs", save_dir / "outputs", dirs_exist_ok=True)
print(f"✅ 已保存到: {save_dir / 'outputs'}")